Lambda School Data Science

*Unit 2, Sprint 3, Module 4*

---


# Model Interpretation 2

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make a Shapley force plot to explain at least 1 individual prediction.
- [ ] Share at least 1 visualization (of any type) on Slack.

But, if you aren't ready to make a Shapley force plot with your own dataset today, that's okay. You can practice this objective with another dataset instead. You may choose any dataset you've worked with previously.

## Stretch Goals
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!


## Links
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
import eli5
from eli5.sklearn import PermutationImportance
from pdpbox.pdp import pdp_isolate, pdp_plot

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
df = sns.load_dataset('titanic').drop(columns=['alive'])
target = 'survived'
features = df.drop(['survived', 'age', 'deck', 'embarked', 'embark_town'], axis = 1).columns

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((668, 9), (223, 9), (668,), (223,))

In [5]:
transformers = make_pipeline(ce.OrdinalEncoder(), SimpleImputer())

X_train = transformers.fit_transform(X_train)
X_test = transformers.transform(X_test)

eval_set = [(X_train, y_train), 
            (X_test, y_test)]

model = XGBRegressor(
    n_estimators=1000, 
    max_depth=10, 
    objective='reg:squarederror', 
    n_jobs=-1, 
)

model.fit(X_train, y_train, eval_set=eval_set, 
          eval_metric='mae', early_stopping_rounds=20)

[0]	validation_0-mae:0.465701	validation_1-mae:0.475604
Multiple eval metrics have been passed: 'validation_1-mae' will be used for early stopping.

Will train until validation_1-mae hasn't improved in 20 rounds.
[1]	validation_0-mae:0.43528	validation_1-mae:0.454747
[2]	validation_0-mae:0.405866	validation_1-mae:0.434612
[3]	validation_0-mae:0.379028	validation_1-mae:0.416562
[4]	validation_0-mae:0.354325	validation_1-mae:0.400153
[5]	validation_0-mae:0.332499	validation_1-mae:0.385877
[6]	validation_0-mae:0.312797	validation_1-mae:0.371614
[7]	validation_0-mae:0.294877	validation_1-mae:0.358916
[8]	validation_0-mae:0.278003	validation_1-mae:0.348064
[9]	validation_0-mae:0.262346	validation_1-mae:0.33776
[10]	validation_0-mae:0.248276	validation_1-mae:0.328385
[11]	validation_0-mae:0.235439	validation_1-mae:0.319827
[12]	validation_0-mae:0.223581	validation_1-mae:0.312091
[13]	validation_0-mae:0.212745	validation_1-mae:0.305705
[14]	validation_0-mae:0.203299	validation_1-mae:0.299997


[141]	validation_0-mae:0.087553	validation_1-mae:0.24021
[142]	validation_0-mae:0.087519	validation_1-mae:0.240201
[143]	validation_0-mae:0.087477	validation_1-mae:0.24017
[144]	validation_0-mae:0.087453	validation_1-mae:0.240165
[145]	validation_0-mae:0.087424	validation_1-mae:0.240173
[146]	validation_0-mae:0.087408	validation_1-mae:0.240172
[147]	validation_0-mae:0.087328	validation_1-mae:0.240166
[148]	validation_0-mae:0.08726	validation_1-mae:0.240141
[149]	validation_0-mae:0.087205	validation_1-mae:0.240125
[150]	validation_0-mae:0.087148	validation_1-mae:0.240101
[151]	validation_0-mae:0.087088	validation_1-mae:0.240075
[152]	validation_0-mae:0.08702	validation_1-mae:0.240067
[153]	validation_0-mae:0.08696	validation_1-mae:0.240052
[154]	validation_0-mae:0.086902	validation_1-mae:0.24004
[155]	validation_0-mae:0.086847	validation_1-mae:0.240036
[156]	validation_0-mae:0.086795	validation_1-mae:0.240034
[157]	validation_0-mae:0.086742	validation_1-mae:0.240032
[158]	validation_0-m

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [6]:
permuter = PermutationImportance(model, scoring='neg_mean_absolute_error', n_iter=3)
permuter.fit(X_test, y_test)

feature_names = df[features].columns.tolist()
eli5.show_weights(permuter, top=None, feature_names = feature_names)

Weight,Feature
0.1682 ± 0.0192,adult_male
0.0392 ± 0.0305,fare
0.0375 ± 0.0154,parch
0.0344 ± 0.0113,pclass
0.0058 ± 0.0043,who
0.0016 ± 0.0012,alone
0 ± 0.0000,class
-0.0003 ± 0.0052,sibsp
-0.0013 ± 0.0041,sex


In [7]:
from sklearn.metrics import roc_auc_score
X_test_processed = processor.transform(X_test)
class_index = 1
y_pred_proba = model.predict_proba(X_test_processed)[:, class_index]
print(f'Test ROC AUC for class {class_index}:')
print(roc_auc_score(y_test, y_pred_proba)) # Ranges from 0-1, higher is better

NameError: name 'processor' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
transformers = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer()
)

X_train = transformers.fit_transform(X_train)
X_test = transformers.transform(X_test)

X_test = pd.DataFrame(X_test, columns = df[features].columns)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
print('Validation Accuracy', rf.score(X_test, y_test))

In [ ]:
X_test.columns

In [ ]:
feature = 'pclass'

isolated = pdp_isolate(
    model=rf, 
    dataset=X_test, 
    model_features=X_test.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature);

In [ ]:
X_test['pclass'].value_counts()

In [ ]:
feature = 'who'

isolated = pdp_isolate(
    model=rf, 
    dataset=X_test, 
    model_features=X_test.columns, 
    feature=feature
)

pdp_plot(isolated, feature_name=feature);

In [ ]:
feature = 'pclass'

encoder = transformers.named_steps['ordinalencoder']
for item in encoder.mapping:
    if item['col'] == feature:
        feature_mapping = item['mapping']
        
feature_mapping = feature_mapping[feature_mapping.index.dropna()]
category_names = feature_mapping.index.tolist()
category_codes = feature_mapping.values.tolist()

isolated = pdp_isolate(
    model=rf, 
    dataset=X_test, 
    model_features=X_test.columns, 
    feature=feature, 
    cust_grid_points=category_codes
)

fig, axes = pdp_plot(isolated, feature_name=feature, 
                     plot_lines=True, frac_to_plot=0.01)

for ax in axes['pdp_ax']:
    ax.set_xticks(category_codes)
    ax.set_xticklabels(category_names)